In [1]:
%%capture
!pip install datasets
!pip install tokenizers
!pip install torchmetrics


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\vmatt\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/931.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/931.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/931.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/931.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/931.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/931.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/931.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/931.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/931.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/931.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/931.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/931.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/931.7 kB

In [ ]:
!pip install -r requirements.txt

In [ ]:
from utils.config import get_config
from train import train_model

cfg = get_config()
train_model(cfg)

Using device: cuda
Device name: NVIDIA A100-SXM4-40GB
Device memory: 39.55743408203125 GB


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence: 309
Max length of target sentence: 274
No model to preload, starting from scratch


Processing Epoch 19: 100%|██████████| 3638/3638 [06:28<00:00,  9.36it/s, loss=2.162]


In [ ]:
from pathlib import Path
import torch
import torch.nn as nn
from utils.config import get_config, latest_weights_file_path, get_weights_file_path
from train import get_model, get_ds, run_validation
# from translate import translate

In [ ]:
import torch

model_filename = "opus_books_weights/tmodel_00.pt"
try:
    state = torch.load(model_filename, map_location="cpu")
    print("Loaded successfully:", state.keys())
except Exception as e:
    print("Error loading model:", str(e))


Loaded successfully: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'global_step'])


In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename =  get_weights_file_path(config,f"19")
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Using device: cuda
Max length of source sentence: 309
Max length of target sentence: 274


<All keys matched successfully>

In [ ]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)

--------------------------------------------------------------------------------
    SOURCE: And George went home again, musing as he walked along, and let himself in.
    TARGET: E Giorgio se ne andò a casa, meditando in cammino.
 PREDICTED: E Giorgio se ne andò a casa , meditando in cammino .
--------------------------------------------------------------------------------
    SOURCE: "It is Julia Severn," replied Miss Temple, very quietly.
    TARGET: — È Giulia Severne, — rispose tranquillamente la direttrice. — Giulia Severne, signora.
 PREDICTED: — È Giulia Severne , — rispose tranquillamente la signorina Temple , con calma . — Giulia Severne , signora .
--------------------------------------------------------------------------------
    SOURCE: I must continue the list.
    TARGET: Facciamo l’elenco.
 PREDICTED: Ho dimenticato la lista .
--------------------------------------------------------------------------------
    SOURCE: Changes in estate also issue from this, for if, to 